# ResNet50 

In [1]:
from keras.applications.vgg19 import VGG19
from keras.models import Model, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, concatenate, Input
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np
from scipy.ndimage import zoom
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from matplotlib import pyplot as plt 

Using TensorFlow backend.


# Create VGG19+ Fully connected layers 

In [2]:
# Import ResNet pretrained model
model = VGG19(weights='imagenet', include_top=False, input_shape = (225, 225, 3))

In [3]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [4]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 225, 225, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 225, 225, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 225, 225, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
def get_images(df):
    images = []
    
    for idx, row in df.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        
        #band_1 *= np.arccos(row['inc_angle'])
        #band_2 *= np.arccos(row['inc_angle'])
        
        # Bilinear interpolation
        band_1 = zoom(band_1, 3, order=1)
        band_2 = zoom(band_2, 3, order=1)
        band_3 = band_2 - band_1
        
        X = np.dstack((band_1, band_2, band_3))

        #X = np.expand_dims(X, axis=0)
        #X = preprocess_input(X)
        
        images.append(X)
        
    return np.array(images)


def get_angles(df):
    angles = []
    max_angle = df['inc_angle'].max()
    
    for idx, row in df.iterrows():
        angle = np.array(row['inc_angle'])
        
        # Pre-Processing:
        #  - ...
        
        #angle /= max_angle
        
        angles.append(angle)
        
    return np.array(angles)


def plot_acc(histobj):
    plt.figure(figsize=(10,10))
    plt.plot(histobj.history['acc'])
    plt.plot(histobj.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    
def plot_loss(histobj):
    plt.figure(figsize=(10,10))
    plt.plot(histobj.history['loss'])
    plt.plot(histobj.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
      
    
def show_image(img):
    fig = plt.figure(figsize=(12, 5))
    ax = plt.subplot(1, 2, 1)
    ax.imshow(img[:, :, 0], cmap=cm.inferno)
    ax.set_title('Band 1')

    ax = plt.subplot(1, 2, 2)
    im = ax.imshow(img[:, :, 1], cmap=cm.inferno)
    ax.set_title('Band 2')
    
    cax = fig.add_axes([0.95, 0.1, 0.03, 0.8])
    fig.colorbar(im, cax=cax, label='[dB]')

    plt.show()
    
    
def get_class(pred, label, img):
    classes = ['ship', 'iceberg']
    pred_i = np.argmax(pred)
    label_i = np.argmax(label)
    print('Prediction class = {}'.format(classes[pred_i]))
    print('Prediction value (%) = {}'.format(pred[pred_i]))
    print('Label class = {}'.format(classes[label_i]))
    show_image(img)

# Import data

In [6]:
train_df = pd.read_json(
    'C:/Users/okarnbla/Documents/private_repos/Kaggle/iceberg_classifier/data/train.json', dtype='float32')
test_df = pd.read_json(
    'C:/Users/okarnbla/Documents/private_repos/Kaggle/iceberg_classifier/data/test.json', dtype='float32')

# Drop N/A

In [7]:
train_df = train_df[train_df['inc_angle'] != 'na']

## Resample 

In [8]:
state = 100
train_ran_df = train_df.sample(frac=1, random_state=state)
train_ran_df = train_ran_df.reset_index(drop=True)

In [9]:
X = get_images(train_ran_df)
print(X.shape)
Y = to_categorical(train_ran_df.is_iceberg.values, num_classes=2) # [0. 1.]=iceberg, [1. 0.]=ship

(1471, 225, 225, 3)


In [10]:
X_test = get_images(test_df)
X_ids = test_df['id']

In [11]:
X[:,0,0,0].shape[0]

1471

## Split train/val data

In [12]:
train_samples = np.round(0.8*(X[:,0,0,0].shape[0]))
train_samples = np.int(train_samples)

In [ ]:
X_train = X[0:train_samples]
Y_train = Y[0:train_samples]

X_val = X[train_samples+1:-1]
Y_val = Y[train_samples+1:-1]

## Train model 

In [ ]:
model_hist = model_final.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=32, epochs=1)

Train on 1177 samples, validate on 292 samples
Epoch 1/1
 608/1177 [==============>...............] - ETA: 815s - loss: 0.8012 - acc: 0.5066

In [ ]:
plot_loss(model_hist)

In [ ]:
plot_acc(model_hist)

In [ ]:
test_preds = model_final.predict(X_test, batch_size=32)

In [ ]:
sample = 30
get_class(val_preds[sample], val_Y[sample], val_X[sample])
is_ice = test_preds[:, 1]
ids = TEST_labels

In [ ]:
with open('subv6.csv', 'w') as fp:
    fp.write('id,is_iceberg\n')
    for i in range(len(X_ids)):
        fp.write('{0:},{1:.10f}\n'.format(X_ids[i], test_preds[i,1]))